<a href="https://colab.research.google.com/github/elylaila/seminario/blob/main/Explainability.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
# Run in Colab to install local packages
%pip install ferret-xai bitsandbytes accelerate numpy==1.23.1
%pip install git+https://github.com/huggingface/transformers.git
# %pip install git+https://github.com/inseq-team/inseq.git

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import pandas as pd
from ferret import Benchmark
import torch
import numpy as np
from tqdm.notebook import tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Seminario/

Mounted at /content/drive
/content/drive/MyDrive/Seminario


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
name = 'dbmdz/bert-base-italian-cased' # inserire bert-ita
model = AutoModelForSequenceClassification.from_pretrained('final_model/checkpoint-4375').to(device) # inserire path modello finale
tokenizer = AutoTokenizer.from_pretrained(name)
print(model.config.id2label)

tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/433 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

{0: 'LABEL_0', 1: 'LABEL_1'}


In [6]:
predictions = pd.read_csv('final_model_predictions.csv', usecols=['y_pred', 'y_true'])
predictions

,y_true,y_pred
0,1,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
995,0,0
996,0,1
997,0,0
998,1,1


In [31]:
df = pd.read_csv('dataset/df_test_clean.csv').drop(columns=['homotransphobic'])
df = pd.concat([df, predictions], axis=1)
df

,text,rationales,y_true,y_pred
0,quasi quasi è meglio femminiello!,"[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 3...",1,0
1,"sotto quel tweet, c'è un ammasso di ritardati ...",NaN,0,0
2,brava mara come sempre puntatona.. sei la zia ...,NaN,0,0
3,copio i gay cagatemi !!! se vi chiedessi di de...,NaN,0,0
4,io: è bono eh papà: ma non eri frocia te,NaN,0,0
...,...,...,...,...
995,rt quindi tu dai dei piglianculo alla gente e ...,NaN,0,0
996,quel sono a lavoro andré che suona un po come ...,NaN,0,1
997,115 gol e 48 assist in 283 presenze. nella top...,NaN,0,0
998,ma non rompermi le balle culattone di merda,"[40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 5...",1,1


In [14]:
from ferret import LIMEExplainer, IntegratedGradientExplainer

In [15]:
exp_lst = [LIMEExplainer(model, tokenizer),
           IntegratedGradientExplainer(
                    model, tokenizer, multiply_by_inputs=False
                )
            ]

In [32]:
bench = Benchmark(model,
                  tokenizer,
                  explainers=exp_lst
                  )

random_state = 42

# List of conditions
conditions = [
    {'y_true_val': 0, 'y_pred_val': 1, 'drop_columns': ['rationales'], 'target': 1},
    {'y_true_val': 1, 'y_pred_val': 1, 'drop_columns': ['rationales'], 'target': 1},
    {'y_true_val': 1, 'y_pred_val': 0, 'drop_columns': ['rationales'], 'target': 0}
]

# Define a function to extract rationales
def get_rationales(rationales_str, frase):
    # Convert string of numbers to a list of integers
    rationales_lst = [int(e) for e in rationales_str[1:-1].split(', ')]

    # Use list comprehension to get characters based on indices
    rationales_ch = ''.join(frase[ch_idx] for ch_idx in rationales_lst)

    return rationales_ch

# Store explanations and rationales for each condition
explanations_dict = {}
rationales_dicts = {}
dfs_samples_dict = {}

for condition in tqdm(conditions):
    # crea i df secondo le condizioni
    filtered_df = df[(df['y_true'] == condition['y_true_val']) & (df['y_pred'] == condition['y_pred_val'])]

    # droppa 'rationales' se necessario
    if condition['drop_columns']:
        filtered_df = filtered_df.drop(columns=condition['drop_columns'])

    # creiamo i sample dei df
    sampled_df = filtered_df.sample(n=5, random_state=random_state)

    # li mettiamo in un dizionario nel caso ci dovessero servire dopo
    df_name = f"df_errors_{condition['y_true_val']}{condition['y_pred_val']}_sample" # il nome viene dato in base alle condizioni sopra
    dfs_samples_dict[df_name] = sampled_df

    # generiamo le spiegazioni per ogni frase nel df, in base al target desiderato, specificato in conditions
    sentences = list(sampled_df['text'])
    explanations = [bench.explain(sentence,
                            target=condition['target']) for sentence in tqdm(sentences)]

    # conserviamo tutto in un dizionario
    explanations_dict[f"explanations_{condition['y_true_val']}{condition['y_pred_val']}"] = explanations

'''
    # se abbiamo la colonna 'rationales' nel df, estriamo la stringa dagli indici
    if 'rationales' in sampled_df:
      rationales_strs = list(sampled_df['rationales'])
      rationales = [get_rationales(rationales_str, sentence) for rationales_str, sentence in zip(rationales_strs, sentences)]
      rationales_dict = dict(zip(sentences, rationales))
      rationales_dicts[f"rationales_{condition['y_true_val']}{condition['y_pred_val']}"] = rationales_dict
'''


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

Explainer:   0%|          | 0/2 [00:00<?, ?it/s]

Explainer:   0%|          | 0/2 [00:00<?, ?it/s]

Explainer:   0%|          | 0/2 [00:00<?, ?it/s]

...

Explainer:   0%|          | 0/2 [00:00<?, ?it/s]

Explainer:   0%|          | 0/2 [00:00<?, ?it/s]

Explainer:   0%|          | 0/2 [00:00<?, ?it/s]

Explainer:   0%|          | 0/2 [00:00<?, ?it/s]

'\n    # se abbiamo la colonna \'rationales\' nel df, estriamo la stringa dagli indici\n    if \'rationales\' in sampled_df:\n      rationales_strs = list(sampled_df[\'rationales\'])\n      rationales = [get_rationales(rationales_str, sentence) for rationales_str, sentence in zip(rationales_strs, sentences)]\n      rationales_dict = dict(zip(sentences, rationales))\n      rationales_dicts[f"rationales_{condition[\'y_true_val\']}{condition[\'y_pred_val\']}"] = rationales_dict\n'

In [35]:
i = 1
exp = explanations_dict['explanations_01'][i]
print(exp[0].text)
bench.show_table(exp)

comunque siamo 5 froci in una piccola macchi a diretti verso una serata frocia


,comunque,siamo,5,fro,##ci,in,una,piccola,macch,##i,a,diretti,verso,una.1,serata,fro.1,##cia
LIME,0.03,-0.00,0.03,0.47,0.03,0.00,-0.04,-0.05,0.04,-0.02,0.04,0.04,-0.02,0.02,-0.07,0.02,-0.07
Integrated Gradient,-0.04,-0.14,-0.08,-0.13,-0.02,-0.07,-0.02,-0.03,-0.05,-0.01,-0.08,-0.04,0.05,-0.06,-0.00,-0.03,-0.09


In [37]:
i = 0
exp = explanations_dict['explanations_11'][i]
print(exp[0].text)
bench.show_table(exp)

ma sei rotto in culo tu che cazzo significa


Token,ma,sei,rotto,in,culo,tu,che,cazzo,significa
LIME,0.27,-0.06,0.20,-0.04,0.18,-0.03,-0.06,0.03,-0.13
Integrated Gradient,-0.19,-0.28,0.11,-0.08,-0.04,0.04,0.02,0.01,0.15


In [41]:
i = 4
exp = explanations_dict['explanations_11'][i]
print(exp[0].text)
bench.show_table(exp)

sai sparare ma sei rotto in culo


Token,sai,sparare,ma,sei,rotto,in,culo
LIME,-0.03,-0.01,0.18,0.01,0.43,0.03,0.30
Integrated Gradient,-0.27,-0.18,0.01,-0.15,-0.06,-0.14,-0.06


In [46]:
s.to_latex(
    column_format="cccccccc", position="h", position_float="centering",
    hrules=True, label="exp", caption="caption",
    multirow_align="t", multicol_align="r"
)

'\\begin{table}[h]\n\\centering\n\\caption{caption}\n\\label{exp}\n\\begin{tabular}{cccccccc}\n\\toprule\nToken & sai & sparare & ma & sei & rotto & in & culo \\\\\n\\midrule\nLIME & \\background-color#ebeef0 \\color#000000 -0.03 & \\background-color#f1f1f2 \\color#000000 -0.01 & \\background-color#eed0d2 \\color#000000 0.18 & \\background-color#f2efef \\color#000000 0.01 & \\background-color#e8a4a9 \\color#000000 0.43 & \\background-color#f2ebeb \\color#000000 0.03 & \\background-color#ebb9bd \\color#000000 0.30 \\\\\nIntegrated Gradient & \\background-color#c0d1de \\color#000000 -0.27 & \\background-color#d1dce5 \\color#000000 -0.18 & \\background-color#f2f1f1 \\color#000000 0.01 & \\background-color#d6e0e7 \\color#000000 -0.15 & \\background-color#e7ebee \\color#000000 -0.06 & \\background-color#d8e1e8 \\color#000000 -0.14 & \\background-color#e7ebee \\color#000000 -0.06 \\\\\n\\bottomrule\n\\end{tabular}\n\\end{table}\n'

In [42]:
bench.get_dataframe(exp)

Token,[CLS],sai,sparare,ma,sei,rotto,in,culo,[SEP]
LIME,0.000000,-0.034390,-0.007517,0.183813,0.009587,0.426962,0.032946,0.304785,0.000000
Integrated Gradient,-0.033187,-0.273655,-0.184726,0.007751,-0.153555,-0.058075,-0.141246,-0.060631,-0.087174


In [ ]:
i = 0
exp = explanations_dict['explanations_10'][i]
print(exp[i].text)
bench.show_table(exp)

@user_abc @user_abcde cazzo ridi che hai una lesbica come punta


,@,us,##er,_,ab,##c,@.1,us.1,##er.1,_.1,ab.1,##c.1,##de,cazzo,ridi,che,hai,una,lesbica,come,punta
Partition SHAP,-0.09,0.08,0.00,0.13,-0.04,0.01,-0.02,0.09,0.06,0.03,-0.00,0.04,0.00,-0.04,0.07,-0.04,0.10,0.04,0.04,0.05,0.02
LIME,-0.03,0.04,0.05,-0.01,-0.03,0.06,-0.08,0.02,0.03,-0.00,-0.01,0.04,-0.00,-0.04,0.08,-0.05,-0.04,0.10,0.12,0.08,0.07
Gradient,0.04,0.02,0.02,0.02,0.02,0.02,0.04,0.02,0.02,0.03,0.02,0.03,0.04,0.10,0.11,0.08,0.05,0.04,0.09,0.03,0.05
Gradient (x Input),-0.18,-0.02,-0.01,0.02,-0.01,0.03,-0.03,0.01,0.02,0.02,-0.01,0.03,-0.01,0.06,0.07,0.14,-0.00,0.03,0.22,-0.01,-0.00
Integrated Gradient,0.02,-0.03,0.04,-0.07,0.02,0.02,-0.05,0.04,0.09,0.04,-0.04,0.03,0.03,-0.03,0.03,0.06,0.07,-0.07,-0.07,-0.08,0.02
Integrated Gradient (x Input),0.00,0.04,0.03,0.04,-0.00,0.02,0.05,0.04,0.01,0.04,0.00,0.03,-0.00,-0.03,-0.02,0.00,-0.03,0.13,0.20,0.23,-0.05


In [ ]:
i = 1
exp = explanations_dict['explanations_10'][i]
print(exp[i].text)
bench.show_table(exp)

È arrivato Kezman a Formello, dopo un summit di circa un paio di ore, ha detto che le piante stanno bene, bisogna da na romanella al muro de cinta, e poi ha detto che siete na banda de cazzari. Io mi fido…. #succhiacazzi #sieteilmaledellalazio


,È,arrivato,Ke,##z,##man,a,For,##mello,",",dopo,un,su,##mmi,##t,di,circa,un.1,paio,di.1,ore,",.1",ha,detto,che,le,piante,stanno,bene,",.2",bisogna,da,na,romane,##lla,al,muro,de,cinta,",.3",e,poi,ha.1,detto.1,che.1,siete,na.1,banda,de.1,ca,##zza,##ri,.,Io,mi,fido,[UNK],..1,#,succhia,##ca,##zzi,#.1,siete.1,##ilm,##ale,##della,##la,##zio
Partition SHAP,0.00,-0.08,0.01,0.00,0.01,0.02,-0.00,-0.01,0.01,0.01,-0.05,0.01,0.02,0.03,-0.02,-0.00,-0.01,0.01,-0.01,-0.03,-0.01,0.01,0.05,0.00,0.01,0.00,0.00,0.07,0.01,0.01,0.01,0.01,0.01,0.00,0.00,-0.00,-0.01,-0.03,-0.00,0.01,0.02,0.01,0.05,-0.00,-0.00,0.00,-0.04,0.00,0.01,0.02,0.04,-0.03,0.02,0.03,0.02,0.02,-0.01,0.01,0.01,0.00,0.00,0.02,0.00,0.01,0.01,0.01,0.00,0.00
LIME,-0.02,-0.04,-0.01,0.01,0.00,0.01,0.03,-0.01,-0.00,0.01,-0.01,-0.01,0.02,0.02,-0.01,-0.00,0.00,0.01,0.00,-0.01,-0.01,-0.01,0.09,0.00,-0.00,0.01,-0.03,0.07,0.00,0.01,-0.00,0.00,0.01,0.00,-0.00,0.00,0.01,-0.01,0.00,0.00,0.03,-0.01,0.05,-0.03,-0.02,0.03,-0.07,-0.03,0.04,0.00,0.00,-0.02,0.01,0.01,-0.02,0.00,-0.01,0.00,0.02,0.01,-0.01,0.01,0.03,0.01,-0.01,0.00,0.01,-0.01
Gradient,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.00,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.03,0.02,0.01,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.02,0.01,0.02,0.01,0.01,0.02,0.02,0.01,0.01,0.02,0.02,0.05,0.02,0.01,0.02,0.01,0.03,0.01,0.01,0.02,0.02,0.03,0.03,0.03,0.01,0.01,0.02,0.01,0.02,0.01,0.01,0.01,0.01
Gradient (x Input),0.02,-0.05,-0.02,-0.01,0.01,-0.01,-0.01,-0.02,-0.01,-0.00,0.01,0.00,0.00,0.01,0.01,0.01,0.01,0.01,0.01,-0.01,-0.01,-0.02,0.02,-0.00,-0.02,0.02,-0.01,0.00,0.01,-0.00,0.00,-0.04,-0.01,-0.00,0.02,0.01,-0.01,-0.00,0.00,0.01,0.03,-0.02,-0.00,-0.01,0.00,-0.03,-0.05,-0.01,-0.00,0.01,-0.01,0.02,-0.00,0.01,0.01,-0.03,0.03,-0.07,-0.03,-0.01,-0.01,-0.06,0.00,0.01,0.00,0.01,0.01,-0.00
Integrated Gradient,0.00,-0.00,-0.02,0.03,0.02,0.02,-0.00,-0.01,0.04,-0.02,-0.00,0.01,0.00,-0.00,0.01,-0.01,0.01,-0.01,-0.01,0.02,0.01,-0.04,-0.02,0.00,-0.01,0.01,0.03,0.02,-0.01,-0.01,-0.01,0.00,0.02,0.02,0.01,0.02,0.01,-0.02,0.00,0.01,-0.01,-0.00,0.04,0.01,-0.04,-0.01,-0.03,0.03,-0.01,-0.02,0.01,-0.01,-0.01,0.01,0.03,-0.01,-0.00,-0.03,-0.02,-0.02,-0.03,0.01,0.02,-0.01,0.02,-0.02,0.00,0.01
Integrated Gradient (x Input),-0.02,0.02,0.01,0.00,-0.01,0.02,0.01,-0.01,-0.00,0.02,0.00,0.00,0.00,0.01,0.00,0.01,0.00,0.01,0.00,0.01,0.00,0.03,0.12,0.03,0.03,-0.00,0.00,0.09,0.01,0.02,-0.01,0.00,-0.00,0.00,0.00,-0.00,0.01,-0.00,0.00,0.00,0.02,0.01,0.07,0.04,0.02,0.00,-0.04,0.02,0.01,0.01,0.01,0.02,0.02,0.01,-0.02,-0.00,0.01,0.02,-0.03,-0.02,-0.01,0.01,-0.01,0.00,0.02,0.00,0.00,-0.01


In [ ]:
i = 2
exp = explanations_dict['explanations_10'][i]
print(exp[i].text)
bench.show_table(exp)

@user_abcdefghij @user_abcde @user_abcde Complimenti per la cultura, la proprietà di linguaggio e il pelo sul petto!! Grande femminiello 🤗🤗🙉🙉


,@,us,##er,_,ab,##c,##de,##f,##ghi,##j,@.1,us.1,##er.1,_.1,ab.1,##c.1,##de.1,@.2,us.2,##er.2,_.2,ab.2,##c.2,##de.2,Complimenti,per,la,cultura,",",la.1,proprietà,di,linguaggio,e,il,pelo,sul,petto,!,!.1,Grande,femmini,##ello,[UNK]
Partition SHAP,-0.03,0.03,0.01,0.03,0.01,0.01,0.00,0.00,-0.03,-0.00,-0.03,0.03,0.01,0.02,-0.01,-0.00,0.01,-0.02,0.02,0.00,0.01,-0.01,0.01,0.01,0.04,0.01,0.03,0.02,-0.02,0.04,0.01,-0.00,0.03,0.00,0.01,0.02,0.02,0.02,0.00,-0.00,0.05,0.23,-0.03,0.06
LIME,-0.00,0.02,0.02,0.00,0.01,0.01,-0.01,-0.03,-0.02,-0.01,-0.02,0.00,0.02,0.03,-0.02,-0.00,-0.02,-0.02,0.00,0.01,-0.00,0.00,0.02,-0.01,0.14,0.02,0.01,0.03,0.02,-0.01,-0.03,-0.03,0.02,0.01,-0.00,0.01,0.01,0.02,0.01,0.01,0.10,-0.02,-0.12,0.09
Gradient,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.03,0.01,0.01,0.02,0.03,0.01,0.02,0.02,0.02,0.02,0.02,0.03,0.02,0.03,0.03,0.05,0.05,0.18,0.06,0.03
Gradient (x Input),0.01,0.01,0.01,-0.00,0.00,-0.00,-0.01,0.02,-0.01,-0.01,-0.00,0.01,0.01,-0.01,-0.00,-0.01,-0.00,0.02,0.01,0.01,0.00,-0.00,-0.01,-0.00,-0.01,-0.01,-0.02,-0.02,0.11,-0.01,0.01,0.01,0.02,-0.00,0.00,-0.03,0.01,-0.06,0.09,0.15,0.03,-0.02,0.14,0.04
Integrated Gradient,-0.02,0.00,-0.01,-0.01,-0.01,0.01,0.01,0.01,0.00,-0.00,-0.01,-0.00,0.00,-0.01,-0.01,-0.00,-0.01,-0.00,-0.01,-0.01,0.01,-0.00,-0.01,-0.00,0.01,-0.03,-0.04,-0.04,-0.01,-0.02,-0.01,0.02,-0.00,0.02,-0.00,-0.02,0.01,-0.06,0.00,0.04,0.08,-0.29,0.07,0.01
Integrated Gradient (x Input),0.00,0.01,0.02,0.01,0.02,0.02,0.01,0.02,-0.02,-0.01,0.01,0.02,0.03,0.01,0.03,0.01,0.04,-0.00,0.01,0.04,0.02,0.02,0.02,0.03,0.02,0.03,0.02,0.06,0.03,0.04,0.03,0.04,0.07,0.01,-0.01,0.03,0.02,0.02,0.06,0.04,-0.02,-0.00,-0.01,-0.01


In [ ]:
i = 3
exp = explanations_dict['explanations_10'][i]
print(exp[i].text)
bench.show_table(exp)

@user_abcd Masturbanti a noi con quella pik da travione😅😂😂😂


Token,@,us,##er,_,ab,##c,##d,Mas,##turba,##nti,a,noi,con,quella,pi,##k,da,[UNK]
Partition SHAP,-0.10,0.05,0.05,-0.00,0.02,0.03,0.04,0.07,0.23,0.01,0.01,-0.01,0.09,0.10,0.01,0.01,0.11,0.06
LIME,-0.01,0.07,0.09,0.02,-0.06,0.02,0.01,-0.04,0.13,0.06,0.00,-0.03,0.13,0.21,0.01,0.06,0.02,-0.02
Gradient,0.07,0.04,0.03,0.04,0.03,0.03,0.03,0.04,0.07,0.03,0.03,0.05,0.04,0.05,0.04,0.06,0.07,0.09
Gradient (x Input),0.07,-0.03,-0.03,0.01,-0.00,-0.02,0.02,-0.08,-0.05,-0.06,-0.03,0.01,-0.02,-0.03,-0.12,-0.03,0.19,-0.07
Integrated Gradient,0.03,-0.01,-0.07,0.02,0.04,-0.01,-0.01,-0.10,0.04,0.01,0.03,-0.15,0.08,-0.15,0.03,0.01,0.06,-0.02
Integrated Gradient (x Input),0.02,0.02,0.05,0.01,-0.00,0.02,0.02,-0.01,-0.01,0.02,0.08,0.24,-0.04,0.31,-0.02,0.12,0.00,-0.01


In [ ]:
i = 4
exp = explanations_dict['explanations_10'][i]
print(exp[i].text)
bench.show_table(exp)

@user_abcdefgh CIAO DONOSO...DI KE PARLI??? 🤔IO NN O POSTATO NESSUN MEME VEDO SOLO UN RATTO KE FUMA UNA SIGARETTA.. SARÀ STATO KUEL CRETINO FEMMINIELLO DI MIO NIPOTE 🤬🤬🤬


,@,us,##er,_,ab,##c,##de,##f,##gh,CIA,##O,DON,##OS,##O.1,.,..1,..2,DI,K,##E,PAR,##LI,?,?.1,?.2,[UNK],N,##N,O,POS,##TA,##TO,NE,##SS,##UN,MEM,##E.1,V,##ED,##O.2,SOL,##O.3,UN,RA,##T,##TO.1,K.1,##E.2,FU,##MA,UNA,SI,##GA,##RE,##T.1,##TA.1,..3,..4,SA,##R,##À,STA,##TO.2,K.2,##UE,##L,CR,##ET,##INO,FE,##M,##MIN,##IE,##LL,##O.4,DI.1,MI,##O.5,N.1,##IP,##OT,##E.3,[UNK].1
Partition SHAP,-0.04,0.03,0.01,0.02,0.02,0.01,-0.00,0.01,-0.01,0.00,0.03,0.06,0.00,0.02,0.01,-0.01,-0.01,-0.00,-0.00,-0.00,0.00,0.00,0.00,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.01,0.01,0.01,0.01,0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.01,0.02,0.00,0.00,0.01,0.01,0.02,-0.00,-0.00,0.10,0.03,0.09,0.05,0.03,-0.01,-0.00,-0.00,-0.01,-0.04,-0.02,0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,0.01,0.00,0.00,0.02,0.01,0.07
LIME,0.02,0.01,-0.01,0.01,-0.01,0.03,-0.00,0.01,-0.01,0.01,0.02,-0.01,-0.00,0.00,-0.00,-0.00,0.00,-0.02,-0.02,-0.01,-0.00,0.01,0.01,0.02,0.00,0.01,0.02,0.01,-0.01,0.02,-0.02,0.00,-0.01,-0.01,0.01,-0.01,-0.01,-0.00,0.01,0.02,0.00,0.00,-0.03,-0.01,-0.00,-0.00,-0.00,0.00,-0.01,0.01,0.04,0.00,-0.01,0.04,0.04,0.00,-0.01,-0.00,0.05,0.01,0.02,0.01,0.03,0.03,-0.02,-0.01,0.00,-0.02,-0.01,-0.01,0.01,0.00,-0.01,-0.01,-0.01,-0.02,-0.01,0.02,-0.01,0.00,0.02,0.01,0.02
Gradient,0.02,0.01,0.00,0.01,0.01,0.01,0.01,0.01,0.01,0.04,0.02,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.02,0.02,0.02,0.01,0.01,0.00,0.00,0.01,0.01,0.01,0.00,0.01,0.01,0.01,0.00,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.00,0.01,0.02,0.02,0.02,0.02,0.04,0.03,0.03,0.02,0.01,0.01,0.01,0.01,0.02,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01,0.01
Gradient (x Input),-0.04,0.01,0.01,-0.02,0.00,-0.00,0.00,-0.00,-0.01,-0.01,0.01,0.01,0.03,0.02,-0.00,-0.00,-0.00,-0.01,0.00,-0.00,-0.00,-0.00,-0.01,-0.03,-0.02,-0.02,0.01,0.00,-0.01,-0.01,-0.01,-0.01,0.00,0.00,0.01,-0.01,0.01,-0.00,0.00,-0.00,0.00,0.01,-0.04,0.01,-0.03,-0.01,-0.01,-0.00,0.01,-0.01,0.00,-0.00,-0.00,0.00,-0.00,-0.01,0.03,0.04,-0.00,-0.01,-0.11,0.05,0.02,0.00,0.00,-0.00,-0.00,-0.02,-0.06,0.00,0.00,-0.00,-0.01,-0.00,0.01,-0.01,0.02,0.00,-0.01,0.00,0.00,0.01,-0.04
Integrated Gradient,-0.02,0.00,0.01,-0.02,-0.01,-0.00,-0.01,-0.01,0.00,-0.02,-0.01,-0.01,-0.01,-0.00,0.01,-0.01,0.01,-0.00,0.02,-0.03,-0.00,0.00,0.02,-0.01,-0.00,-0.00,-0.01,-0.01,-0.01,-0.01,0.01,-0.01,-0.00,0.00,-0.02,-0.01,0.00,-0.00,-0.01,0.00,0.02,0.00,-0.00,0.00,-0.01,0.03,-0.01,-0.06,-0.02,0.02,-0.08,0.00,0.02,0.01,-0.00,0.03,0.00,0.02,-0.04,-0.00,-0.02,-0.01,0.00,-0.01,-0.00,0.00,0.01,0.00,-0.01,0.00,-0.01,-0.07,-0.00,0.00,0.00,-0.04,0.01,0.02,0.01,0.00,-0.03,-0.01,0.01
Integrated Gradient (x Input),-0.01,0.00,0.01,0.01,0.01,0.01,0.01,0.00,0.01,0.00,0.00,-0.01,0.00,-0.02,0.01,0.01,0.01,0.00,0.03,0.02,0.01,-0.01,0.02,0.00,0.00,-0.00,-0.01,-0.00,-0.01,0.01,-0.00,-0.01,-0.00,-0.00,0.00,0.00,0.00,0.02,0.03,-0.01,0.01,0.01,-0.00,-0.02,0.01,0.00,0.00,-0.02,-0.00,-0.01,0.04,-0.01,-0.01,0.00,-0.00,0.00,0.00,0.02,0.03,0.02,0.01,-0.01,0.01,0.01,0.01,-0.03,0.00,-0.00,-0.04,0.01,0.02,0.06,0.03,0.02,-0.04,-0.04,0.03,-0.03,-0.01,-0.01,0.02,0.00,-0.01


In [ ]:
rationales_dict

{'@user_abcdefghi ma sei rotto in culo tu che cazzo significa': 'ma sei rotto in culo tu',
 '@user_abcdef Certo è un noto culattone @user_abcdefghij la carriera l‘ha fatta con Arpa testa bassa e culo alzato': 'Certo è un noto culattone @user_abcdefghijtesta bassa e culo alzato',
 '@user_abcd CHI ? AHHH QUESTO COGLIONE RICCHIONE SODOMITA PEDOFILO… 😂… HA ANCHE LA TESTA A FORMA DI GLANDE. STAI ATTENTO PELATO, CHE SIAMO IN TANTI A PISCIARTI IN TESTA, TI RICRESCONO I CAPELLI POI… NON ABBOCCATE SONO BALLE. IL PRIMO SCHIAFFO LO PRENDE LUI. 🎪🤡🤡🤡': 'QUESTO COGLIONE RICCHIONE SODOMITA PEDOFILO… 😂… HA ANCHE LA TESTA A FORMA DI GLANDEIL PRIMO SCHIAFFO LO PRENDE LUI. 🎪🤡🤡🤡',
 "@user_abcdefgh Questi hanno tutta la mia stima perché hanno fatto il culo ai nazi rottinculo di Azov. Per i quali le nullità in servizio permanente come Gramellini trovavano le giustificazioni tipiche della merda che elargiscono con cura maniacale. Attaccati fortissimo al caxxo! Per l'eternità! [URL]": 'ai nazi rottinculo di A